In [1]:
!git clone https://hf-mirror.com/sentence-transformers/all-MiniLM-L6-v2

正克隆到 'all-MiniLM-L6-v2'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 112 (delta 44), reused 53 (delta 19), pack-reused 0 (from 0)
接收对象中: 100% (112/112), 359.04 KiB | 2.90 MiB/s, 完成.
处理 delta 中: 100% (44/44), 完成.
过滤内容: 100% (15/15), 930.41 MiB | 13.19 MiB/s, 完成.


In [2]:
!git clone https://hf-mirror.com/GanymedeNil/text2vec-base-chinese

正克隆到 'text2vec-base-chinese'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 20 (from 1)
展开对象中: 100% (23/23), 161.11 KiB | 2.40 MiB/s, 完成.


In [3]:
!git clone https://hf-mirror.com/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

正克隆到 'paraphrase-multilingual-MiniLM-L12-v2'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 100 (delta 25), reused 0 (delta 0), pack-reused 41 (from 1)
接收对象中: 100% (100/100), 3.45 MiB | 8.77 MiB/s, 完成.
处理 delta 中: 100% (38/38), 完成.
过滤内容: 100% (17/17), 4.30 GiB | 9.61 MiB/s, 完成.


In [1]:
pip install langchain langchain-community faiss-cpu pandas openpyxl

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 80.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.5 MB/s eta 0:00:00
    torch (>=1.9.*)
           ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os
import yaml
import json
from typing import List, Dict, Any
from dataclasses import dataclass, field
import logging

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---- 配置类 ----
@dataclass
class RAGConfig:
    """RAG 管道配置参数"""
    # 嵌入模型配置
    embedding_model: str = "text2vec-base-chinese"
    # embedding_model: str ="all-MiniLM-L6-v2"
    # 检索配置
    search_top_k: int = 3
    search_type: str = "similarity"
    
    # 模型服务配置
    llm_model_name: str = "Qwen/QwQ-32B"
    llm_api_base: str = "https://api-inference.modelscope.cn/v1/"
    llm_streaming: bool = True
    llm_temperature: float = 0.3
    
    # 提示模板
    prompt_template: str = """基于以下上下文信息，请以专业的方式回答问题。如果无法从上下文中获得答案，请明确说明。
    
    上下文：
    {context}
    
    问题：{question}
    
    请用中文提供结构清晰的回答："""
    
    # 文档处理配置
    chunk_size: int = 500
    chunk_overlap: int = 50
    
    # 缓存配置
    cache_file_path: Path = field(default_factory=lambda: Path("vector_store_cache.faiss"))
    
    # 安全配置
    allow_dangerous_deserialization: bool = True
    
    # 调试配置
    verbose: bool = True

    def __post_init__(self) -> None:
        """参数验证"""
        if self.search_top_k <= 0:
            raise ValueError("search_top_k 必须为正整数")
        if self.chunk_overlap >= self.chunk_size:
            raise ValueError("chunk_overlap 不能大于 chunk_size")
        if not (0 <= self.llm_temperature <= 2):
            raise ValueError("temperature 必须在 0-2 之间")

    @classmethod
    def from_yaml(cls, file_path: Path) -> "RAGConfig":
        """从 YAML 文件加载配置"""
        try:
            with open(file_path, 'r') as f:
                config_data = yaml.safe_load(f)
            
            # 确保 cache_file_path 是 Path 对象
            if isinstance(config_data.get('cache_file_path'), str):
                config_data['cache_file_path'] = Path(config_data['cache_file_path'])
            
            return cls(**config_data)
        except Exception as e:
            logger.error(f"YAML 配置加载失败: {e}")
            raise

    @classmethod
    def from_json(cls, file_path: Path) -> "RAGConfig":
        """从 JSON 文件加载配置"""
        try:
            with open(file_path, 'r') as f:
                config_data = json.load(f)
            
            # 确保 cache_file_path 是 Path 对象
            if isinstance(config_data.get('cache_file_path'), str):
                config_data['cache_file_path'] = Path(config_data['cache_file_path'])
            
            return cls(**config_data)
        except Exception as e:
            logger.error(f"JSON 配置加载失败: {e}")
            raise

    def to_dict(self) -> Dict[str, Any]:
        """转换为字典（用于持久化）"""
        return self.__dict__

# ---- 嵌入模型封装 ----
class SentenceTransformerEmbeddings(Embeddings):
    """自定义句子嵌入模型封装"""
    def __init__(self, config: RAGConfig) -> None:
        self.model = SentenceTransformer(config.embedding_model)
        logger.info(f"初始化嵌入模型: {config.embedding_model}")

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text).tolist()

# ---- 模块初始化函数 ----
def initialize_embeddings(config: RAGConfig) -> SentenceTransformerEmbeddings:
    """初始化嵌入模型"""
    return SentenceTransformerEmbeddings(config)

def initialize_vector_store(
    texts: List[str], 
    embeddings: Embeddings,
    config: RAGConfig
) -> FAISS:
    """初始化向量数据库"""
    try:
        # 检查缓存文件是否存在
        if config.cache_file_path.exists():
            logger.warning("注意：正在启用不安全的反序列化选项。确保你信任缓存文件的来源！")
            vector_store = FAISS.load_local(
                str(config.cache_file_path.parent), 
                embeddings, 
                index_name=config.cache_file_path.stem, 
                allow_dangerous_deserialization=config.allow_dangerous_deserialization
            )
        else:
            logger.info("正在创建向量存储...")
            vector_store = FAISS.from_texts(
                texts=texts,
                embedding=embeddings,
                metadatas=[{"source": f"doc_{i}"} for i in range(len(texts))]
            )
            # 保存到缓存文件
            vector_store.save_local(str(config.cache_file_path.parent), index_name=config.cache_file_path.stem)
            logger.info(f"向量存储已保存到缓存文件: {config.cache_file_path}")
        return vector_store
    except Exception as e:
        logger.error(f"向量存储初始化失败: {e}")
        raise

def initialize_llm(config: RAGConfig) -> ChatOpenAI:
    """初始化大语言模型"""
    # api_key = os.getenv("OPENAI_API_KEY", "XXXXXXX")
    api_key = os.getenv("OPENAI_API_KEY")
    
    # api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("未找到 OPENAI_API_KEY 环境变量")

    return ChatOpenAI(
        openai_api_base=config.llm_api_base,
        openai_api_key=api_key,
        model_name=config.llm_model_name,
        streaming=config.llm_streaming,
        temperature=config.llm_temperature
    )

def build_qa_chain(
    vector_store: FAISS,
    llm: ChatOpenAI,
    config: RAGConfig
) -> RetrievalQA:
    """构建检索问答链"""
    # 配置检索器
    retriever = vector_store.as_retriever(
        search_type=config.search_type,
        search_kwargs={"k": config.search_top_k}
    )
    
    # 构建提示模板
    prompt = PromptTemplate(
        template=config.prompt_template,
        input_variables=["context", "question"]
    )
    
    # 配置问答链
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": prompt,
            "verbose": config.verbose
        }
    )
import pandas as pd

def read_excel_qa_pairs(file_paths: List[str]) -> List[str]:
    corpus = []
    seen = set()
    for file_path in file_paths:
        df = pd.read_excel(file_path, sheet_name="Sheet1")
        for _, row in df.iterrows():
            text = f"问题：{row['问题']}\n答案：{row['答案']}"
            if text not in seen:
                corpus.append(text)
                seen.add(text)
    # print(len(corpus))
    return corpus
    
def main(config: RAGConfig) -> None:
    """RAG 主流程"""
    # 读取Excel文件
    excel_files = ["全网运行指标问题.xlsx", "数字人问答.xlsx"]
    corpus = read_excel_qa_pairs(excel_files)
    
    try:
        # 初始化各组件
        embeddings = initialize_embeddings(config)
        vector_store = initialize_vector_store(corpus, embeddings, config)
        llm = initialize_llm(config)
        qa_chain = build_qa_chain(vector_store, llm, config)
        
        # 执行查询
        query = "数据中心是什么？"
        logger.info(f"正在处理查询: {query}")
        
        # 获取完整响应
        response = qa_chain({"query": query})
        
        # 输出结果
        print("\n=== 最终回答 ===")
        print(response["result"])
        
        # 显示参考来源
        if config.verbose:
            print("\n=== 参考文档 ===")
            for doc in response["source_documents"]:
                print(f"- {doc.page_content[:80]}...")

    except Exception as e:
        logger.error(f"流程执行失败: {e}")
        raise

if __name__ == "__main__":
    # 使用方式 1: 默认配置
    # main(RAGConfig())
    
    # # 使用方式 2: 从 YAML 加载配置
    config = RAGConfig.from_yaml(Path("config.yml"))
    main(config)
    
    # # 使用方式 3: 从 JSON 加载配置
    # config = RAGConfig.from_json(Path("config.json"))
    # main(config)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:初始化嵌入模型: all-MiniLM-L6-v2
INFO:__main__:正在创建向量存储...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

INFO:__main__:向量存储已保存到缓存文件: vector_store_cache.faiss
INFO:__main__:正在处理查询: 数据中心是什么？


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
基于以下上下文信息，请以专业的方式回答问题。如果无法从上下文中获得答案，请明确说明。

    上下文：
    问题：什么是核心网？
答案：我们可以把核心网络想象成移动通信系统的“大脑”或“交通指挥中心”。它负责管理手机用户的通话、短信、上网数据，并确保这些信息正确、安全地传递到目的地（比如另一个手机、互联网服务器或银行系统）。它是连接用户设备与外部世界（如互联网或其他运营商网络）的关键枢纽，同时也是实现各种通信服务的核心所在。

问题：数据中心规模区分
答案：大规模数据中心规模巨大，拥有10000个以上标准机架；中型规模数据中心规模适中，拥有1000至3000个标准机架；边缘计算数据中心规模小，通常为小型或微型

问题：核心网为什么重要？
答案：核心网之所以重要，是因为它不仅决定了用户体验的质量，还影响着整个网络的安全性和效率。例如，在紧急情况下，如果核心网不能快速响应，则可能延误救援行动；同样地，如果没有良好的安全措施来保护核心网免受攻击，则可能导致敏感信息泄露。

    问题：数据中心是什么？

    请用中文提供结构清晰的回答：


INFO:httpx:HTTP Request: POST https://api-inference.modelscope.cn/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.

> Finished chain.

=== 最终回答 ===
根据提供的上下文信息，**无法直接获取“数据中心是什么”的明确定义**。上下文仅提供了关于数据中心规模的分类（大规模、中型、边缘计算数据中心）以及核心网的功能和重要性，但未对“数据中心”的概念进行基础性解释。

若您需要进一步定义，可参考通用知识：  
**数据中心**是用于存储、处理和分发大量数据的物理或虚拟设施，通常包含服务器、存储设备、网络设备、供电系统及冷却系统等，旨在为用户提供数据计算、存储、传输和应用服务。它是现代数字化基础设施的核心，支撑着云计算、大数据、人工智能等技术的运行，并与核心网等通信系统协同工作，确保数据高效安全地传递至用户或外部网络。  

建议补充相关上下文以获得更精准的回答。

=== 参考文档 ===
- 问题：什么是核心网？
答案：我们可以把核心网络想象成移动通信系统的“大脑”或“交通指挥中心”。它负责管理手机用户的通话、短信、上网数据，并确保这些信息正确、安全...
- 问题：数据中心规模区分
答案：大规模数据中心规模巨大，拥有10000个以上标准机架；中型规模数据中心规模适中，拥有1000至3000个标准机架；边缘计算数据中心...
- 问题：核心网为什么重要？
答案：核心网之所以重要，是因为它不仅决定了用户体验的质量，还影响着整个网络的安全性和效率。例如，在紧急情况下，如果核心网不能快速响应，...
